In [12]:
# Loading Packages
library(data.table);
library(dplyr);
library(reshape);
library(stringdist)

rename = dplyr::rename

In [49]:
TourneySeeds <- fread("../../data/TourneySeeds.csv")
SampleSubmission <- fread("../../data/SampleSubmission.csv")
Seasons <- fread("../../data/Seasons.csv")
Teams <- fread("../../data/Teams.csv")
TourneySlots <- fread("../../data/TourneySlots.csv")
TourneyDetailedResults <- fread("../../data/TourneyDetailedResults.csv")
#TourneyCompactResults <- fread("data/TourneyCompactResults.csv")
RegularSeasonDetailedResults <- fread("../../data/RegularSeasonDetailedResults.csv")
#RegularSeasonCompactResults <- fread("data/RegularSeasonCompactResults.csv")
KenPom <- fread("../../data/KenPom.csv")
TeamSpelling <- fread("../../data/TeamSpellings.csv")

In [14]:
print(TeamSpelling)

         name_spelling team_id
  1:  a&m-corpus chris    1394
  2:       abilene chr    1101
  3: abilene christian    1101
  4: abilene-christian    1101
  5:         air force    1102
 ---                          
988:              yale    1463
989:     youngstown st    1464
990:    youngstown st.    1464
991:  youngstown state    1464
992:  youngstown-state    1464


In [15]:
print(TourneySeeds)

      Season Seed Team
   1:   1985  W01 1207
   2:   1985  W02 1210
   3:   1985  W03 1228
   4:   1985  W04 1260
   5:   1985  W05 1374
  ---                 
2078:   2016  Z13 1423
2079:   2016  Z14 1453
2080:   2016  Z15 1167
2081:   2016 Z16a 1221
2082:   2016 Z16b 1380


In [16]:
TeamSpelling <- rbind(TeamSpelling,rename(Teams,name_spelling=Team_Name,team_id=Team_Id))
# Extracting seeds for each team
TourneySeeds <- TourneySeeds %>%
    mutate(SeedNum = gsub("[A-Z+a-z]", "", Seed)) %>% select(Season, Team, SeedNum)

In [17]:
print(TeamSpelling)

          name_spelling team_id
   1:  a&m-corpus chris    1394
   2:       abilene chr    1101
   3: abilene christian    1101
   4: abilene-christian    1101
   5:         air force    1102
  ---                          
1352:         Wright St    1460
1353:           Wyoming    1461
1354:            Xavier    1462
1355:              Yale    1463
1356:     Youngstown St    1464


In [18]:
print(TourneySeeds)

      Season Team SeedNum
   1:   1985 1207      01
   2:   1985 1210      02
   3:   1985 1228      03
   4:   1985 1260      04
   5:   1985 1374      05
  ---                    
2078:   2016 1423      13
2079:   2016 1453      14
2080:   2016 1167      15
2081:   2016 1221      16
2082:   2016 1380      16


In [50]:
print(KenPom)

      Year Rank                 Team Conference Wins Losses Seed   Pyth AdjustO
   1: 2002    1                 Duke        ACC   31      4    1 0.9689   117.7
   2: 2002    2             Maryland        ACC   32      4    1 0.9551   117.5
   3: 2002    3           Cincinnati       CUSA   31      4    1 0.9508   115.8
   4: 2002    4               Kansas        B12   31      4    1 0.9482   116.0
   5: 2002    5             Oklahoma        B12   31      5    2 0.9352   112.7
  ---                                                                          
5098: 2016  347 UT Rio Grande Valley        WAC    6     19   NA 0.0847    94.6
5099: 2016  348        Grambling St.       SWAC    5     18   NA 0.0700    88.2
5100: 2016  349         Delaware St.       MEAC    5     20   NA 0.0642    87.8
5101: 2016  350          Florida A&M       MEAC    7     16   NA 0.0628    86.7
5102: 2016  351  Central Connecticut        NEC    3     22   NA 0.0574    91.5
      AdjustO Rank AdjustD AdjustD Rank 

In [19]:
# fix external data
colnames(TeamSpelling)[1] = "Team"
KenPom$Team = tolower(KenPom$Team)
Teams$Team_Name = tolower(Teams$Team_Name)

# mamp spelling
KenPom = left_join(KenPom, TeamSpelling, by=c('Team'='Team'))

In [20]:
print(KenPom)

Source: local data table [5,102 x 25]

                Team  Year  Rank Conference  Wins Losses  Seed   Pyth AdjustO
               (chr) (int) (int)      (chr) (int)  (int) (int)  (dbl)   (dbl)
1  abilene christian  2014   348       Slnd    11     20    NA 0.0793    94.6
2  abilene christian  2015   335       Slnd    10     21    NA 0.1101    92.5
3  abilene christian  2016   316       Slnd    11     13    NA 0.1728    95.0
4          air force  2002   179        MWC     9     19    NA 0.4412   101.1
5          air force  2003   171        MWC    12     16    NA 0.4667   102.9
6          air force  2004    43        MWC    22      7    11 0.8111   108.3
7          air force  2005    49        MWC    18     12    NA 0.7811   109.3
8          air force  2006    47        MWC    24      7    13 0.7940   110.7
9          air force  2007    28        MWC    26      9    NA 0.8649   112.6
10         air force  2008   158        MWC    16     14    NA 0.5064    98.3
..               ...   ..

In [21]:
winner_Tdata = select(RegularSeasonDetailedResults, Season, Wteam, Wscore, Wfgm:Wpf)
loser_Tdata = select(RegularSeasonDetailedResults, Season, Lteam, Lscore, Lfgm:Lpf)
Team_Game_History = rbind(winner_Tdata, setNames(loser_Tdata, names(winner_Tdata)))
colnames(Team_Game_History) = c("season",
                                "team",
                                "score",
                                "fgm",
                                "fga",
                                "fgm3",
                                "fga3",
                                "ftm",
                                "fta",
                                "or",
                                "dr",
                                "ast",
                                "to",
                                "stl",
                                "blk",
                                "Wpf")

In [22]:
team_data_by_season = Team_Game_History %>% group_by(season, team) %>% summarise_each(funs(mean))

team1_data_by_season = data.frame(team_data_by_season)
colnames(team1_data_by_season) <- paste("team1", colnames(team1_data_by_season), sep = "_")
team2_data_by_season = data.frame(team_data_by_season)
colnames(team2_data_by_season) <- paste("team2", colnames(team2_data_by_season), sep = "_")

team1_kenpom_data_by_season = data.frame(KenPom)
colnames(team1_kenpom_data_by_season) <- paste("team1", colnames(team1_kenpom_data_by_season), sep = "_")
team2_kenpom_data_by_season = data.frame(KenPom)
colnames(team2_kenpom_data_by_season) <- paste("team2", colnames(team2_kenpom_data_by_season), sep = "_")

In [40]:
################################################################################
# Get going with actual machine learning:
# 1. set train,test,predict
# 2. add available data
# 3. build models
# 4. Make Predictions
################################################################################
# Prepare prediction target
games.to.train <- RegularSeasonDetailedResults %>%
                  mutate(season=Season, team1=Wteam, team2=Lteam, Score_diff=Wscore-Lscore, team1win=1) %>%
                  select(season, team1, team2, Score_diff, team1win)
games.to.test <- TourneyDetailedResults %>%
                  mutate(season=Season, team1=Wteam, team2=Lteam, Score_diff=Wscore-Lscore, team1win=1) %>%
                  select(season, team1, team2, Score_diff, team1win)
games.to.predict <- cbind(SampleSubmission$Id, colsplit(SampleSubmission$Id, split = "_", names = c('season', 'team1', 'team2')))

flippedGames = function(game){
  flipped <- game %>%
  rename(team1=team2,team2=team1) %>%
  mutate(Score_diff=-Score_diff, team1win=0)
}

games.to.train = rbind(games.to.train,flippedGames(games.to.train))

# Add available data to each target game
addDataToGames = function(games) {

  games <- data.frame(games) %>%
          # add March madness seed to teams in games
          left_join(TourneySeeds, by=c("season"="Season", "team1"="Team")) %>%
          rename(team1seed = SeedNum) %>%
          left_join(TourneySeeds, by=c("season"="Season", "team2"="Team")) %>%
          rename(team2seed = SeedNum) %>%
          mutate(team1seed = as.numeric(team1seed), team2seed = as.numeric(team2seed)) %>%
          # add seasonal data
          left_join(team1_data_by_season,by=c("season" = "team1_season", "team1"="team1_team")) %>%
          left_join(team2_data_by_season,by=c("season" = "team2_season", "team2"="team2_team")) %>%
          # add external data
          left_join(team1_kenpom_data_by_season,by=c("season" = "team1_Year", "team1"="team1_team_id")) %>%
          left_join(team2_kenpom_data_by_season,by=c("season" = "team2_Year", "team2"="team2_team_id"))
  return(games)

}
games.to.predict

Warning message:
In `[<-.factor`(`*tmp*`, ri, value = "⋮"): invalid factor level, NA generatedWarning message:
In `[<-.factor`(`*tmp*`, ri, value = "⋮"): invalid factor level, NA generated

,SampleSubmission$Id,season,team1,team2
1,2016_1112_1114,2016,1112,1114
2,2016_1112_1122,2016,1112,1122
3,2016_1112_1124,2016,1112,1124
4,2016_1112_1138,2016,1112,1138
5,2016_1112_1139,2016,1112,1139
6,2016_1112_1143,2016,1112,1143
7,2016_1112_1151,2016,1112,1151
8,2016_1112_1153,2016,1112,1153
9,2016_1112_1160,2016,1112,1160
10,2016_1112_1163,2016,1112,1163


In [41]:
games.to.train = addDataToGames(games.to.train)
games.to.test = addDataToGames(games.to.test)
games.to.predict = addDataToGames(games.to.predict)

In [44]:
games.to.predict

Warning message:
In `[<-.factor`(`*tmp*`, ri, value = "⋮"): invalid factor level, NA generatedWarning message:
In `[<-.factor`(`*tmp*`, ri, value = "⋱"): invalid factor level, NA generatedWarning message:
In `[<-.factor`(`*tmp*`, ri, value = "⋮"): invalid factor level, NA generatedWarning message:
In `[<-.factor`(`*tmp*`, ri, value = "⋱"): invalid factor level, NA generated

,SampleSubmission.Id,season,team1,team2,team1seed,team2seed,team1_score,team1_fgm,team1_fga,team1_fgm3,ellip.h,team2_Luck,team2_Luck.Rank,team2_SOS.Pyth,team2_SOS.Pyth.Rank,team2_SOS.OppO,team2_SOS.OppO.Rank,team2_SOS.OppD,team2_SOS.OppD.Rank,team2_NCSOS.Pyth,team2_NCSOS.Pyth.Rank
1,2016_1112_1114,2016,1112,1114,6,12,81.2121212121212,28.0606060606061,58.2424242424242,6.51515151515152,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,2016_1112_1122,2016,1112,1122,6,16,81.2121212121212,28.0606060606061,58.2424242424242,6.51515151515152,⋯,0.006,158,0.4539,218,104.5,116,106.2,320,0.5438,143
3,2016_1112_1124,2016,1112,1124,6,5,81.2121212121212,28.0606060606061,58.2424242424242,6.51515151515152,⋯,0.045,79,0.6687,28,105.2,84,99,4,0.4295,266
4,2016_1112_1138,2016,1112,1138,6,14,81.2121212121212,28.0606060606061,58.2424242424242,6.51515151515152,⋯,0.058,54,0.5531,100,106,61,104,194,0.5635,108
5,2016_1112_1139,2016,1112,1139,6,9,81.2121212121212,28.0606060606061,58.2424242424242,6.51515151515152,⋯,-0.022,228,0.5968,76,104.3,131,100.8,40,0.4172,283
6,2016_1112_1143,2016,1112,1143,6,4,81.2121212121212,28.0606060606061,58.2424242424242,6.51515151515152,⋯,-0.059,304,0.6218,58,106.2,52,101.7,73,0.4541,237
7,2016_1112_1151,2016,1112,1151,6,12,81.2121212121212,28.0606060606061,58.2424242424242,6.51515151515152,⋯,0.103,15,0.4832,182,104.9,101,105.5,281,0.549,136
8,2016_1112_1153,2016,1112,1153,6,9,81.2121212121212,28.0606060606061,58.2424242424242,6.51515151515152,⋯,-0.049,289,0.5458,109,104.3,129,102.6,112,0.4608,228
9,2016_1112_1160,2016,1112,1160,6,8,81.2121212121212,28.0606060606061,58.2424242424242,6.51515151515152,⋯,0.05,71,0.6072,73,106.5,45,102.5,105,0.441,252
10,2016_1112_1163,2016,1112,1163,6,9,81.2121212121212,28.0606060606061,58.2424242424242,6.51515151515152,⋯,-0.064,311,0.5672,92,104.8,107,102.3,98,0.4781,216


In [48]:
e=Teams$Team_Name[which(Teams$Team_Id %in% unique(games.to.predict$team2[which(is.na(games.to.predict$team2_Team))]))]
f=Teams$Team_Name[which(Teams$Team_Id %in% unique(games.to.predict$team1[which(is.na(games.to.predict$team1_Team))]))]
missing = unique(c(e,f))
print(missing)
class(missing)

[1] "ark little rock" "cs bakersfield" 


[1] "character"

In [24]:
games.to.train = addDataToGames(games.to.train)
games.to.test = addDataToGames(games.to.test)
games.to.predict = addDataToGames(games.to.predict)

games.to.train = games.to.train %>% na.omit()
games.to.test = games.to.test %>% na.omit()

getMissing = function(){
  a=Teams$Team_Name[which(Teams$Team_Id %in% unique(games.to.test$team2[which(is.na(games.to.test$team2_Team))]))]
  b=Teams$Team_Name[which(Teams$Team_Id %in% unique(games.to.test$team1[which(is.na(games.to.test$team1_Team))]))]
  c=Teams$Team_Name[which(Teams$Team_Id %in% unique(games.to.train$team2[which(is.na(games.to.train$team2_Team))]))]
  d=Teams$Team_Name[which(Teams$Team_Id %in% unique(games.to.train$team1[which(is.na(games.to.train$team1_Team))]))]
  e=Teams$Team_Name[which(Teams$Team_Id %in% unique(games.to.predict$team2[which(is.na(games.to.predict$team2_Team))]))]
  f=Teams$Team_Name[which(Teams$Team_Id %in% unique(games.to.predict$team1[which(is.na(games.to.predict$team1_Team))]))]
  missing = unique(c(a,b,c,d,e,f))
  print(missing)
  return(missing)
}

m.score_diff <- lm(Score_diff~ .,
                   data=select(games.to.train,
                               -c(team1win,
                                  team1_Team,
                                  team2_Team,
                                  team1,
                                  team2)))

games.to.train$Predicted_Score_diff = predict(m.score_diff)
games.to.test$Predicted_Score_diff = predict(m.score_diff,games.to.test)
games.to.predict$Predicted_Score_diff = predict(m.score_diff,games.to.predict)

win_chance = ecdf(games.to.train$Score_diff)

games.to.train$Pred = win_chance(games.to.train$Predicted_Score_diff)
games.to.test$Pred = win_chance(games.to.test$Predicted_Score_diff)
games.to.predict$Pred = win_chance(games.to.predict$Predicted_Score_diff)

Warning message:
In predict.lm(m.score_diff, games.to.test): prediction from a rank-deficient fit may be misleadingWarning message:
In predict.lm(m.score_diff, games.to.predict): prediction from a rank-deficient fit may be misleading

In [25]:
getLogLoss = function(games){
  y = games$team1win
  pred = games$Pred
  logLoss = -mean(y*log(pred) + (1-y)*log(1-pred))
  return(logLoss)
}

getLogLoss(games.to.train)

getLogLoss(games.to.test)

write.csv(games.to.predict %>% select(Id=SampleSubmission.Id, Pred), 'seed_submission.csv', row.names=FALSE)

[1] 0.5957008

[1] 0.5340887

In [32]:
games.to.predict

Warning message:
In `[<-.factor`(`*tmp*`, ri, value = "⋮"): invalid factor level, NA generatedWarning message:
In `[<-.factor`(`*tmp*`, ri, value = "⋱"): invalid factor level, NA generatedWarning message:
In `[<-.factor`(`*tmp*`, ri, value = "⋮"): invalid factor level, NA generatedWarning message:
In `[<-.factor`(`*tmp*`, ri, value = "⋱"): invalid factor level, NA generated

,SampleSubmission.Id,season,team1,team2,team1seed,team2seed,team1_score,team1_fgm,team1_fga,team1_fgm3,ellip.h,team2_SOS.Pyth,team2_SOS.Pyth.Rank,team2_SOS.OppO,team2_SOS.OppO.Rank,team2_SOS.OppD,team2_SOS.OppD.Rank,team2_NCSOS.Pyth,team2_NCSOS.Pyth.Rank,Predicted_Score_diff,Pred
1,2016_1112_1114,2016,1112,1114,6,12,81.2121212121212,28.0606060606061,58.2424242424242,6.51515151515152,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,2016_1112_1122,2016,1112,1122,6,16,81.2121212121212,28.0606060606061,58.2424242424242,6.51515151515152,⋯,0.4539,218,104.5,116,106.2,320,0.5438,143,13.5323797955906,0.861111111111111
3,2016_1112_1124,2016,1112,1124,6,5,81.2121212121212,28.0606060606061,58.2424242424242,6.51515151515152,⋯,0.6687,28,105.2,84,99,4,0.4295,266,-2.43832906438329,0.443763616557734
4,2016_1112_1138,2016,1112,1138,6,14,81.2121212121212,28.0606060606061,58.2424242424242,6.51515151515152,⋯,0.5531,100,106,61,104,194,0.5635,108,8.69795990225478,0.743055555555556
5,2016_1112_1139,2016,1112,1139,6,9,81.2121212121212,28.0606060606061,58.2424242424242,6.51515151515152,⋯,0.5968,76,104.3,131,100.8,40,0.4172,283,-0.246919423621096,0.5
6,2016_1112_1143,2016,1112,1143,6,4,81.2121212121212,28.0606060606061,58.2424242424242,6.51515151515152,⋯,0.6218,58,106.2,52,101.7,73,0.4541,237,-0.967625057125517,0.5
7,2016_1112_1151,2016,1112,1151,6,12,81.2121212121212,28.0606060606061,58.2424242424242,6.51515151515152,⋯,0.4832,182,104.9,101,105.5,281,0.549,136,4.38951555952007,0.627723311546841
8,2016_1112_1153,2016,1112,1153,6,9,81.2121212121212,28.0606060606061,58.2424242424242,6.51515151515152,⋯,0.5458,109,104.3,129,102.6,112,0.4608,228,1.57447488332754,0.523556644880174
9,2016_1112_1160,2016,1112,1160,6,8,81.2121212121212,28.0606060606061,58.2424242424242,6.51515151515152,⋯,0.6072,73,106.5,45,102.5,105,0.441,252,3.95188482580975,0.594498910675381
10,2016_1112_1163,2016,1112,1163,6,9,81.2121212121212,28.0606060606061,58.2424242424242,6.51515151515152,⋯,0.5672,92,104.8,107,102.3,98,0.4781,216,1.27565297807452,0.523556644880174
